## Taking Totals from xlsx files

In [1]:
balance_code = "OVERHEAD"
import os
import pandas as pd
import json
from rapidfuzz import fuzz
import re
from collections import defaultdict

data = json.load(open('C:/Users/Civic Homes/Desktop/Jobs/path.json'))
JobCode = json.load(open('C:/Users/Civic Homes/Desktop/Jobs/JobCode.json'))

construction = pd.read_csv("costCentreConstruction_GL.csv")
construction_col = construction.columns


# Cleaning up dir
def clean_dir(dir):
    dir_split = dir.split(" ")
    if dir_split[0].lower() != "lot":
        lot_num = dir_split[0].strip("L,l")
        name = dir_split[1]

        return lot_num + " " + name
    
    return dir_split[1] + " " + dir_split[2]
    

# Finding cost centers
def cost_center_finder(cost_center, total):
    if len(cost_center) == 0:
        # Match cost center and give a score
        cost_center_num_probable, score = match_2_list(total)
        # If a score is above a certain level
        if score >= 90:
            cost_center_num = cost_center_num_probable
            GL_account = construction[construction["code"] == cost_center_num]["GLAccount"].values[0]

            return cost_center_num, GL_account
        else:
            cost_center_num = instructions_from_estimators(total)
            
            if cost_center_num:
                GL_account = construction[construction["code"] == cost_center_num]["GLAccount"].values[0]
                return cost_center_num, GL_account 
            
            else:
                return "unknown", "unknown"
        
    else:
        # Obtain cost center from job code/cost center
        cost_center_num_raw = cost_center[0][0].split('/')[-1]
        # Remove point symbol from back
        cost_center_num = re.sub("\.[0-9]+", "", cost_center_num_raw)
        
        # check if cost center num exists
        if cost_center_num in list(construction["code"]):
            GL_account = construction[construction["code"] == cost_center_num]["GLAccount"].values[0]
            return cost_center_num, GL_account
        
        else:
            cost_center_num_probable, score = match_2_list(total)
            if score >= 90:
                cost_center_num = cost_center_num_probable
                GL_account = construction[construction["code"] == cost_center_num]["GLAccount"].values[0]

                return cost_center_num, GL_account
            else:
                cost_center_num = instructions_from_estimators(total)
            
                if cost_center_num:
                    GL_account = construction[construction["code"] == cost_center_num]["GLAccount"].values[0]
                    return cost_center_num, GL_account 

                else:
                    return "unknown", "unknown"

# Manually changing cost centres number for an entry
def instructions_from_estimators(total):
    total_dict = {
        "Total for Misc. Payments - Frame Stage": 375,
        "Total for Misc. Payments - Base Stage": 375,
        "Total for Services - Electricity/Water/D. Insurance": 100,
        "Total for Pre-Fabricated Wall Frames": 300,
        "Total for STRUCTURAL STEEL": 330,
        "Total for Misc. Payments - Fixing Stage": 960,
        #"Total for Misc. Payments - Lock up Stage": 490,
        #"Total for Subcontractor":,
        #"Total for Materials":,
        #"Total for ELECTRICITY/WATER":seperate,
        
    }
    
    if total in total_dict:
        return total_dict[total]
    
    return False
    

def str_cleaning(str_raw):
    str_lower = str_raw.lower()

    str_clean = re.sub('total for ', '', str_lower)

    str_ultra_clean = re.sub(' / ', '/', str_clean)

    return str_ultra_clean

def match_2_str(str1, str2):
    str1_clean = str_cleaning(str1)
    str2_clean = str_cleaning(str2)

    return fuzz.ratio(str1_clean, str2_clean)

def match_2_list(str1):
    fuzz_score_dict = {}
    
    for cost_center in construction[construction_col[2]]:
        fuzz_score = match_2_str(str1, cost_center)
        fuzz_score_dict[fuzz_score] = cost_center
        
    category = fuzz_score_dict[sorted(fuzz_score_dict)[-1]]
    return construction[construction[construction_col[2]] == category]["code"].values[0], sorted(fuzz_score_dict)[-1]

def find_job_code(file):
    job_name = re.sub('.xlsx','',file)
    lot_num = re.findall('[0-9]+', job_name)[0]
    name = re.findall('[A-Za-z]+', job_name)[0]

    for address in JobCode["Address"]:
        address_lower = address.lower()
        if str(lot_num).lower() in address_lower.split(" ") and str(name).lower() in address_lower.split(" "):
            job_code = JobCode["Address"][address][2]

            return job_code, address
        
    return file, file

list_files = [file for file in os.listdir("C:\\Users\\Civic Homes\\Desktop\\Jobs\\JobsJournalEntries") if file != ".ipynb_checkpoints"]


print(list_files)
for file in list_files:
    GL_credit = defaultdict(float)
    if file == ".ipynb_checkpoints" or "~" in list(file):
        continue
    print(file)
    try:
        df = pd.read_excel("C:\\Users\\Civic Homes\\Desktop\\Jobs\\JobsJournalEntries\\"+ file, header = 4)
    except:
        print('failed')
        continue
    
    df = df.rename(columns = {'Unnamed: 0': 'Category'})
    
    date = '31/12/2022'
    
    category_list = list(df['Category'])

    totals_list = [total for total in category_list if re.findall("Total for ", str(total))]
    
    clean_file = clean_dir(file)
    
    job_code, address = find_job_code(str(clean_file))

    array = []
    credit_amount = 0
    for num, total in enumerate(totals_list):
        category = re.sub('Total for ', '', total)

        start = df[df["Category"] == category].index[0]
        end = df[df["Category"] == total].index[0]

        category_df = df.iloc[start:end + 1, :]

        cost_center = [re.findall('[ UH0-9]+\/[0-9.]+', str(account)) for account in category_df["Memo/Description"] if len(re.findall('[ UH0-9]+\/[0-9\.]+', str(account))) != 0]
         
        cost_center_num, GL_account = cost_center_finder(cost_center, total)

        amount = category_df[category_df["Category"] == total]["Amount"].values[0]
        
        debit = amount
        credit = 0

        array.append([job_code, "OB", "", cost_center_num, 0 ,credit, debit, date, date, date, GL_account, False, f"{total}", ""])
        GL_credit[GL_account] += debit
        
        
    # Adding credit line
    for GL_account in GL_credit:
        if GL_account == "unknown":
            continue
        array.append([balance_code, "OB", "", "JNL", 0 ,GL_credit[GL_account], 0, date, date, date, GL_account, False, f"Balancing GL {GL_account}", ""])
        
        
    csv_df = pd.DataFrame(array).set_index(0)
    bsv_df =  pd.DataFrame(array, columns = ["Job Code", "OB", "Empty", "code", "Empty2", "Credit", "Debit", "Date", "Date2", "Date3", "GL_account", "False", "Comments", "Empty3"])
    if len(bsv_df[bsv_df["code"] == "unknown"]) != 0:
        display(bsv_df[bsv_df["code"] == "unknown"])
        
    file_name = re.sub("\.xlsx", "", clean_file)
    file_name = re.sub(" ", "_", file_name)
    bsv_df_sorted = bsv_df.sort_values(by = ["GL_account", "Job Code", "Debit"], ascending = [True, False, False])
    #csv_df.to_csv(f"C:\\Users\\Civic Homes\\Desktop\\Jobs\\JobsJournalEntries\\jnl_files\\{file_name}.jnl", sep = "|", header = False)
    bsv_df_sorted = bsv_df_sorted.set_index("Job Code")
    bsv_df_sorted.to_csv(f"C:\\Users\\Civic Homes\\Desktop\\Jobs\\JobsJournalEntries\\jnl_files\\{file_name}.jnl", sep = "|", header = False)

['101 Barolo.xlsx', '1012 Hansel.xlsx', '102 Barolo.xlsx', '103 Barolo.xlsx', '104 Alisma.xlsx', '105 Alisma.xlsx', '108 Alisma.xlsx', '110 Tanninbrae.xlsx', '1102 Derrimut.xlsx', '1141 Kevin.xlsx', '122 Dollarbird.xlsx', '1249 Almon.xlsx', '127 Litoria.xlsx', '1270 Trellis.xlsx', '132 Skyros.xlsx', '138 Litoria.xlsx', '1407 Binap.xlsx', '1447 Brightvale.xlsx', '15 Subzero.xlsx', '1537 Jevons.xlsx', '156 Litoria.xlsx', '157 Litoria.xlsx', '1617 Fegent.xlsx', '17 Lakes.xlsx', '176 Fenfield.xlsx', '1802 Azas.xlsx', '1819 Yellowfinn.xlsx', '19 Opal.xlsx', '215 Alkira.xlsx', '215 Carnation.xlsx', '33 Grandview.xlsx', '561 Tarneit.xlsx', '60 Halpin.xlsx', '628 Harriot.xlsx', '6361 Edith.xlsx', '761 Axle.xlsx', 'jnl_detailed_files', 'jnl_files', 'L1, Medway.xlsx', 'L219, Maton Street.xlsx', 'L222, Alister Grove.xlsx', 'L2909 Hogans Road.xlsx', 'L32, Grandview Cr.xlsx', 'L34, Grandview Cr..xlsx', 'L352, Walton Street.xlsx', 'L405, Dennison St.xlsx', 'L427 Pascal way.xlsx', 'L43, Hampshire Ave

c:\Users\Civic Homes\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\Civic Homes\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,Job Code,OB,Empty,code,Empty2,Credit,Debit,Date,Date2,Date3,GL_account,False,Comments,Empty3
10,U04582,OB,,unknown,0,0.0,130.84,31/12/2022,31/12/2022,31/12/2022,unknown,False,Total for Misc. Payments - Lock up Stage,


c:\Users\Civic Homes\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\Civic Homes\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


102 Barolo.xlsx
103 Barolo.xlsx
104 Alisma.xlsx
105 Alisma.xlsx
108 Alisma.xlsx
110 Tanninbrae.xlsx
1102 Derrimut.xlsx


...

L222, Alister Grove.xlsx
L2909 Hogans Road.xlsx
L32, Grandview Cr.xlsx
L34, Grandview Cr..xlsx
L352, Walton Street.xlsx


,Job Code,OB,Empty,code,Empty2,Credit,Debit,Date,Date2,Date3,GL_account,False,Comments,Empty3
29,U 04731,OB,,unknown,0,0.0,3262.70,31/12/2022,31/12/2022,31/12/2022,unknown,False,Total for Materials,
33,U 04731,OB,,unknown,0,0.0,207.01,31/12/2022,31/12/2022,31/12/2022,unknown,False,Total for Misc. Payments - Lock up Stage,


L405, Dennison St.xlsx
L427 Pascal way.xlsx
L43, Hampshire Avenue.xlsx
L430, Aaron Road.xlsx
L517 Sabro.xlsx
L518, Bruno Av.xlsx
L526, Sabro Street.xlsx
L636, Willowleaf Street.xlsx
L6705 Scoparia Street.xlsx
L716, Flycatcher Road.xlsx
L884 Scorpio.xlsx
L929 Romney.xlsx


,Job Code,OB,Empty,code,Empty2,Credit,Debit,Date,Date2,Date3,GL_account,False,Comments,Empty3
18,U04957,OB,,unknown,0,0.0,67.69,31/12/2022,31/12/2022,31/12/2022,unknown,False,Total for Misc. Payments - Lock up Stage,


Lot 6407 Window Road.xlsx


## More detailed using compositions of a total rather than total

In [2]:
import os
import pandas as pd
import json
from rapidfuzz import fuzz
import re
from collections import defaultdict

data = json.load(open('C:/Users/Civic Homes/Desktop/Jobs/path.json'))
JobCode = json.load(open('C:/Users/Civic Homes/Desktop/Jobs/JobCode.json'))

construction = pd.read_csv("costCentreConstruction_GL.csv")

# Cleaning up dir
def clean_dir(dir):
    dir_split = dir.split(" ")
    if dir_split[0].lower() != "lot":
        lot_num = dir_split[0].strip("L,l")
        name = dir_split[1]

        return lot_num + " " + name
    
    return dir_split[1] + " " + dir_split[2]
    

# Finding cost centers
def cost_center_finder(cost_center, total):
    if len(cost_center) == 0:
        # Match cost center and give a score
        cost_center_num_probable, score = match_2_list(total)
        # If a score is above a certain level
        if score >= 90:
            cost_center_num = cost_center_num_probable
            GL_account = construction[construction["code"] == cost_center_num]["GLAccount"].values[0]

            return cost_center_num, GL_account
        else:
            cost_center_num = instructions_from_estimators(total)
            
            if cost_center_num:
                GL_account = construction[construction["code"] == cost_center_num]["GLAccount"].values[0]
                return cost_center_num, GL_account 
            
            else:
                return "unknown", "unknown"
        
    else:
        # Obtain cost center from job code/cost center
        cost_center_num_raw = cost_center[0][0].split('/')[-1]
        # Remove point symbol from back
        cost_center_num = re.sub("\.[0-9]+", "", cost_center_num_raw)
        
        # check if cost center num exists
        if cost_center_num in list(construction["code"]):
            GL_account = construction[construction["code"] == cost_center_num]["GLAccount"].values[0]
            return cost_center_num, GL_account
        
        else:
            cost_center_num_probable, score = match_2_list(total)
            if score >= 90:
                cost_center_num = cost_center_num_probable
                GL_account = construction[construction["code"] == cost_center_num]["GLAccount"].values[0]

                return cost_center_num, GL_account
            else:
                cost_center_num = instructions_from_estimators(total)
            
                if cost_center_num:
                    GL_account = construction[construction["code"] == cost_center_num]["GLAccount"].values[0]
                    return cost_center_num, GL_account 

                else:
                    return "unknown", "unknown"

# Manually changing cost centres number for an entry
def instructions_from_estimators(total):
    total_dict = {
        "Total for Misc. Payments - Frame Stage": 375,
        "Total for Misc. Payments - Base Stage": 375,
        "Total for Services - Electricity/Water/D. Insurance": 100,
        "Total for Pre-Fabricated Wall Frames": 300,
        "Total for STRUCTURAL STEEL": 330,
        "Total for Misc. Payments - Fixing Stage": 960,
        #"Total for Misc. Payments - Lock up Stage": 490,
        #"Total for Subcontractor":,
        #"Total for Materials":,
        #"Total for ELECTRICITY/WATER":seperate,
        
    }
    
    if total in total_dict:
        return total_dict[total]
    
    return False
    

def str_cleaning(str_raw):
    str_lower = str_raw.lower()

    str_clean = re.sub('total for ', '', str_lower)

    str_ultra_clean = re.sub(' / ', '/', str_clean)

    return str_ultra_clean

def match_2_str(str1, str2):
    str1_clean = str_cleaning(str1)
    str2_clean = str_cleaning(str2)

    return fuzz.ratio(str1_clean, str2_clean)

def match_2_list(str1):
    fuzz_score_dict = {}
    
    for cost_center in construction[construction_col[2]]:
        fuzz_score = match_2_str(str1, cost_center)
        fuzz_score_dict[fuzz_score] = cost_center
        
    category = fuzz_score_dict[sorted(fuzz_score_dict)[-1]]
    return construction[construction[construction_col[2]] == category]["code"].values[0], sorted(fuzz_score_dict)[-1]

def find_job_code(file):
    job_name = re.sub('.xlsx','',file)
    lot_num = re.findall('[0-9]+', job_name)[0]
    name = re.findall('[A-Za-z]+', job_name)[0]

    for address in JobCode["Address"]:
        address_lower = address.lower()
        if str(lot_num).lower() in address_lower.split(" ") and str(name).lower() in address_lower.split(" "):
            job_code = JobCode["Address"][address][2]

            return job_code, address
        
    return file, file

list_files = [file for file in os.listdir("C:\\Users\\Civic Homes\\Desktop\\Jobs\\JobsJournalEntries") if file != ".ipynb_checkpoints"]


print(list_files)
for file in list_files:
    GL_credit = defaultdict(float)
    if file == ".ipynb_checkpoints" or "~" in list(file):
        continue
    print(file)
    try:
        df = pd.read_excel("C:\\Users\\Civic Homes\\Desktop\\Jobs\\JobsJournalEntries\\"+ file, header = 4)
    except:
        print('failed')
        continue
    
    df = df.rename(columns = {'Unnamed: 0': 'Category'})
    
    date = '31/12/2022'
    
    category_list = list(df['Category'])

    totals_list = [total for total in category_list if re.findall("Total for ", str(total))]
    
    clean_file = clean_dir(file)
    
    job_code, address = find_job_code(str(clean_file))

    array = []
    credit_amount = 0
    for num, total in enumerate(totals_list):
        category = re.sub('Total for ', '', total)

        start = df[df["Category"] == category].index[0]
        end = df[df["Category"] == total].index[0]

        category_df = df.iloc[start:end, :]

        cost_center = [re.findall('[ UH0-9]+\/[0-9.]+', str(account)) for account in category_df["Memo/Description"] if len(re.findall('[ UH0-9]+\/[0-9\.]+', str(account))) != 0]
         
        cost_center_num, GL_account = cost_center_finder(cost_center, total)
        
        # Taking each specific cost
        for index in range(1, len(category_df)):
            amount = category_df.iloc[index, :]["Amount"]
            description = category_df.iloc[index, :]["Memo/Description"]
            account = category_df.iloc[index, :]["Account"]
            try:
                name = category_df.iloc[index, :]["Name"]
                name_total = f" ; {name}"
            except:
                name_total = ""
                pass

            debit = amount
            credit = 0

            array.append([job_code, "OB", "", cost_center_num, 0 ,credit, debit, date, date, date, GL_account, False, f"{description} ; {account}{name_total}", ""])
            GL_credit[GL_account] += debit
        
        
    # Adding credit line
    for GL_account in GL_credit:
        if GL_account == "unknown":
            continue
        array.append([balance_code, "OB", "", "JNL", 0 ,GL_credit[GL_account], 0, date, date, date, GL_account, False, f"Balancing GL {GL_account}", ""])
        
        
    csv_df = pd.DataFrame(array).set_index(0)
    bsv_df =  pd.DataFrame(array, columns = ["Job Code", "OB", "Empty", "code", "Empty2", "Credit", "Debit", "Date", "Date2", "Date3", "GL_account", "False", "Comments", "Empty3"])
    
    file_name = re.sub("\.xlsx", "", clean_file)
    file_name = re.sub(" ", "_", file_name)
    bsv_df_sorted = bsv_df.sort_values(by = ["GL_account", "Job Code", "Debit"], ascending = [True, False, False])
    #csv_df.to_csv(f"C:\\Users\\Civic Homes\\Desktop\\Jobs\\JobsJournalEntries\\jnl_files\\{file_name}.jnl", sep = "|", header = False)
    bsv_df_sorted = bsv_df_sorted.set_index("Job Code")
    bsv_df_sorted.to_csv(f"C:\\Users\\Civic Homes\\Desktop\\Jobs\\JobsJournalEntries\\jnl_detailed_files\\{file_name}_detailed.jnl", sep = "|", header = False)

    if len(bsv_df[bsv_df["code"] == "unknown"]) != 0:
        display(bsv_df[bsv_df["code"] == "unknown"])
        

['101 Barolo.xlsx', '1012 Hansel.xlsx', '102 Barolo.xlsx', '103 Barolo.xlsx', '104 Alisma.xlsx', '105 Alisma.xlsx', '108 Alisma.xlsx', '110 Tanninbrae.xlsx', '1102 Derrimut.xlsx', '1141 Kevin.xlsx', '122 Dollarbird.xlsx', '1249 Almon.xlsx', '127 Litoria.xlsx', '1270 Trellis.xlsx', '132 Skyros.xlsx', '138 Litoria.xlsx', '1407 Binap.xlsx', '1447 Brightvale.xlsx', '15 Subzero.xlsx', '1537 Jevons.xlsx', '156 Litoria.xlsx', '157 Litoria.xlsx', '1617 Fegent.xlsx', '17 Lakes.xlsx', '176 Fenfield.xlsx', '1802 Azas.xlsx', '1819 Yellowfinn.xlsx', '19 Opal.xlsx', '215 Alkira.xlsx', '215 Carnation.xlsx', '33 Grandview.xlsx', '561 Tarneit.xlsx', '60 Halpin.xlsx', '628 Harriot.xlsx', '6361 Edith.xlsx', '761 Axle.xlsx', 'jnl_detailed_files', 'jnl_files', 'L1, Medway.xlsx', 'L219, Maton Street.xlsx', 'L222, Alister Grove.xlsx', 'L2909 Hogans Road.xlsx', 'L32, Grandview Cr.xlsx', 'L34, Grandview Cr..xlsx', 'L352, Walton Street.xlsx', 'L405, Dennison St.xlsx', 'L427 Pascal way.xlsx', 'L43, Hampshire Ave

c:\Users\Civic Homes\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\Civic Homes\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,Job Code,OB,Empty,code,Empty2,Credit,Debit,Date,Date2,Date3,GL_account,False,Comments,Empty3
20,U04582,OB,,unknown,0,0.0,130.84,31/12/2022,31/12/2022,31/12/2022,unknown,False,BUNNINGS BILL - REIMBURSEMENT ; Misc. Payments...,


102 Barolo.xlsx
103 Barolo.xlsx
104 Alisma.xlsx
105 Alisma.xlsx


c:\Users\Civic Homes\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\Civic Homes\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\Civic Homes\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\Civic Homes\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


...

L222, Alister Grove.xlsx
L2909 Hogans Road.xlsx
L32, Grandview Cr.xlsx
L34, Grandview Cr..xlsx
L352, Walton Street.xlsx


,Job Code,OB,Empty,code,Empty2,Credit,Debit,Date,Date2,Date3,GL_account,False,Comments,Empty3
50,U 04731,OB,,unknown,0,0.0,3262.70,31/12/2022,31/12/2022,31/12/2022,unknown,False,Supply & delivery aluminium windows ; Material...,
55,U 04731,OB,,unknown,0,0.0,34.50,31/12/2022,31/12/2022,31/12/2022,unknown,False,Drill Bit/Nails/Packer Window Macsin Mix ; Mis...,
56,U 04731,OB,,unknown,0,0.0,85.07,31/12/2022,31/12/2022,31/12/2022,unknown,False,Nail Collated ; Misc. Payments - Lock up Stage...,
57,U 04731,OB,,unknown,0,0.0,87.44,31/12/2022,31/12/2022,31/12/2022,unknown,False,Drill Bit/ Masonry Solu. misc. materials ; Mis...,


L405, Dennison St.xlsx
L427 Pascal way.xlsx
L43, Hampshire Avenue.xlsx
L430, Aaron Road.xlsx
L517 Sabro.xlsx
L518, Bruno Av.xlsx
L526, Sabro Street.xlsx
L636, Willowleaf Street.xlsx
L6705 Scoparia Street.xlsx
L716, Flycatcher Road.xlsx
L884 Scorpio.xlsx
L929 Romney.xlsx


,Job Code,OB,Empty,code,Empty2,Credit,Debit,Date,Date2,Date3,GL_account,False,Comments,Empty3
41,U04957,OB,,unknown,0,0.0,30.55,31/12/2022,31/12/2022,31/12/2022,unknown,False,BUNNINGS BILL - REIMBURSEMENT ; Misc. Payments...,
42,U04957,OB,,unknown,0,0.0,37.14,31/12/2022,31/12/2022,31/12/2022,unknown,False,BUNNINGS BILL - REIMBURSEMENT ; Misc. Payments...,


Lot 6407 Window Road.xlsx
